In [1]:
import os
import pandas as pd

In [2]:
BASE_DATA = 'data'
PREPROCESSED_DATA = 'preprocessed_data'

MOTHES = ['Mar', 'May', 'Jul', 'Sep']

In [3]:
for filename in os.listdir(BASE_DATA):
    if not filename.startswith("Futures prices"):
        continue
    df = pd.read_excel(os.path.join(BASE_DATA, filename))
    df['Date'] = pd.to_datetime(df['Date'], format='%d %b %Y')  # FIXME
    columns = df.columns
    
    comodadity = filename.split()[4]  # FIXME
    if not os.path.exists(f"{PREPROCESSED_DATA}/{comodadity}"):
        os.makedirs(f"{PREPROCESSED_DATA}/{comodadity}")
        
    first_year = df['Date'].dt.year.min()
    last_year = df['Date'].dt.year.max()
    
    for year in range(first_year, last_year + 1):
        year_df = df[df['Date'].dt.year == year]
        rel_columns =  [col + ' ' + str(year + 1) for col in MOTHES]
        year_df = year_df[['Date'] + rel_columns]
        
        # column with the data diffrence between the current date and the previous date, the first has diff = 0
        year_df['diff'] = year_df['Date'].diff().dt.days.fillna(1).astype(int)
        
        year_df.to_csv(f"{PREPROCESSED_DATA}/{comodadity}/{comodadity}_{year}.csv", index=False)
        
        # check if there are missing data
        assert year_df.isna().sum().sum() == 0, f"Missing data in {comodadity}_{year}"
        